In [1]:
from dataloc import *
from bgs_helpers import *
from astropy.table import Table

In [ ]:
# Creates SV3 FUJI merged file (no Y3 supplement yet)
create_merged_file(BGS_SV3_ANY_FULL_FILE, IAN_BGS_SV3_MERGED_NOY3_FILE, 'sv3', photoz_wspec=False)

# Now remove galaxies in the patches of SV3 that have poor overlap with Y3
sv3_table: Table = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')
print(len(sv3_table))

sv3_table['region'] = tile_to_region(sv3_table['NEAREST_TILEIDS'][:,0])
to_remove = np.isin(sv3_table['region'], sv3_poor_y3overlap)
sv3_table.remove_rows(to_remove)
print(len(sv3_table))

sv3_table.write(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits', overwrite='True')

In [ ]:
# Creates Y1 IRON merged file
create_merged_file(BGS_Y1_ANY_FULL_FILE, IAN_BGS_Y1_MERGED_FILE, "1")

In [ ]:
# Creates Y3 LOA merged file
create_merged_file(BGS_Y3_ANY_FULL_FILE, IAN_BGS_Y3_MERGED_FILE_LOA, "3")

# Add columns to allow the possibility to make a Y3 catalog that is cut to SV3 regions.
table = Table.read(IAN_BGS_Y3_MERGED_FILE_LOA, format='fits')
sv3tiles = read_tiles_Y3_sv3()
ntiles_inside, nearest_tile_ids = find_tiles_for_galaxies(sv3tiles, table_to_df(table), 10)
if 'NTILE_MINE_SV3' in table.columns:
    table.remove_columns(['NTILE_MINE_SV3'])
if 'NEAREST_TILEIDS_SV3' in table.columns:
    table.remove_columns(['NEAREST_TILEIDS_SV3'])
table.add_column(ntiles_inside, name="NTILE_MINE_SV3")
table.add_column(nearest_tile_ids, name="NEAREST_TILEIDS_SV3")

table.write(IAN_BGS_Y3_MERGED_FILE_LOA, format='fits', overwrite=True)

In [ ]:
# Now add Y3 galaxies to supplement the NN catalog, especialy valuable at the edges of the regions
# They won't go into the main catalog because their NTILE_MINE is < 10
supplement_sv3_merged_file_with_y3(IAN_BGS_SV3_MERGED_NOY3_FILE, IAN_BGS_Y3_MERGED_FILE_LOA, IAN_BGS_SV3_MERGED_FILE)

In [ ]:
# Make a variant of Y3 Loa cut to SV3 regions, for the Fiber Incompleteness Study

# First replace photo-z column with one that has no spec-z 'contamination' 
table = add_photz_columns(IAN_BGS_Y3_MERGED_FILE_LOA, IAN_PHOT_Z_FILE_NOSPEC)

# Remove galaxies that are far away from the SV3 regions
keep = table['NTILE_MINE_SV3'] >= 1 # Not going to 10+ here because we want to keep nearby galaxies for the NN catalog

print(f"Y3 Loa table cut from {len(table)} to {keep.sum()} rows for like-SV3 version")
table  = table[keep]

table.write(IAN_BGS_Y3_MERGED_FILE_LOA_SV3CUT, format='fits', overwrite=True)


Adding photo-z columns to table.
Z_PHOT already in table, replacing it.
Phot-z file has phot-z for 99.95% of targets.
17858989
18987204
17858989
<TableColumns names=('TARGETID','Z','ZWARN','SPECTYPE','DELTACHI2','NTILE','TILES','RA','DEC','FLUX_G','FLUX_R','MASKBITS','PROB_OBS','DN4000','DN4000_IVAR','DN4000_MODEL','DN4000_MODEL_IVAR','ABSMAG01_SDSS_G','ABSMAG01_SDSS_G_IVAR','ABSMAG01_SDSS_R','ABSMAG01_SDSS_R_IVAR','SFR','SFR_IVAR','LOGMSTAR','LOGMSTAR_IVAR','HALPHA_EW','HALPHA_EW_IVAR','HBETA_EW','HBETA_EW_IVAR','MORPHTYPE','FRACFLUX_G','FRACFLUX_R','FRACFLUX_Z','SHAPE_E1','SHAPE_E2','SHAPE_R_IVAR','SHAPE_E1_IVAR','SHAPE_E2_IVAR','SERSIC','SERSIC_IVAR','SURVEY','PROGRAM','APP_MAG_R','APP_MAG_G','ABS_MAG_R','ABS_MAG_R_K','ABS_MAG_G','ABS_MAG_G_K','LOG_L_GAL','G_R_BEST','QUIESCENT','NTILE_MINE','NEAREST_TILEIDS','MATCH_DIST_1','NTILE_MINE_SV3','NEAREST_TILEIDS_SV3','Z_PHOT','RELEASE','BRICKID','OBJID','MATCH_DIST_2','REF_CAT')>
Y3 Loa table cut from 17858989 to 254100 rows for like-SV3 